In [8]:
%%capture # this will hide the cell output
!pip install pandas
!pip install numpy
!pip install plotly
!pip install sklearn

# Lets start with all of the imports
import pandas as pd
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import plotly
import plotly.graph_objs as go 
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [9]:

# Now lets input the methods
# LINEAR REGRESSION #
#####################
def get_linear_pred(DF_DATASET):
    X = DF_DATASET[['year','month','day','dayofyear']]
    n_y = DF_DATASET[['n_extent']]
    s_y = DF_DATASET[['s_extent']]

    N_L_REGR = LinearRegression(normalize=True).fit(X, n_y)
    S_L_REGR = LinearRegression(normalize=True).fit(X, s_y)
    return N_L_REGR, S_L_REGR

def get_prediction(month_, day_, year_, DF_DATASET, N_L_REGR, S_L_REGR):
    date_to_pred = str(month_).zfill(2)+"-"+str(day_).zfill(2)+"-"+str(year_)# '03-01-2022'
    day_to_predict = Dummy(date_to_pred)
    # list of day of the year values + or - 15 days
    add_subtract_list = get_add_subtract_days(month_, day_, year_, 15)
    df_data = DF_DATASET.loc[(DF_DATASET['dayofyear'].isin(add_subtract_list))]
    # this is will select days that are within the date range + or - 10 days
    X = df_data[['year','month','day','dayofyear']]
    n_y = df_data[['n_extent']]
    s_y = df_data[['s_extent']]
    # North Model Creation and test
    X_train, X_test, y_train, y_test = train_test_split(X, n_y, test_size=0.3, shuffle=True)
    n_l_regr = LinearRegression(normalize=True).fit(X_train, y_train)
    n_score = n_l_regr.score(X_test, y_test)

    # South Model Creation and test
    X_train, X_test, y_train, y_test = train_test_split(X, s_y, test_size=0.3, shuffle=True)
    s_l_regr = LinearRegression(normalize=True).fit(X_train, y_train)
    s_score = s_l_regr.score(X_test, y_test)

    # create an object for the prediction output
    _d = {'year': [int(year_)],
      'month': [int(month_)],
      'day' : [int(day_)],
     'dayofyear': day_of_year_getter(day_to_predict)[0]}
    # pass that object for the prediction
    north = N_L_REGR.predict(pd.DataFrame(data=_d))
    south = S_L_REGR.predict(pd.DataFrame(data=_d))
    return north, south, n_score, s_score
# CLUSTERING #
##############
def scale_data(DF_DATASET):
    scaler = StandardScaler()
    scaler.fit(DF_DATASET)
    scaled_data = scaler.transform(DF_DATASET)
    return scaled_data
def encode_labels(DF_DATASET, encode=True):
    # if encode is false, then undo the encoding
    try:
        if encode == False:
            # unencode
            DF_DATASET['date'] = le.inverse_transform(DF_DATASET['date'])
        else : # else encode the labels
            # encode
            le.fit(DF_DATASET['date'])
            DF_DATASET['date'] = le.transform(DF_DATASET['date'])
    except:
        # this means that the average dataframe has been passed in
        # inelegant, but it works...
        if encode == False:
            # unencode
            DF_DATASET['hemisphere'] = le.inverse_transform(DF_DATASET['hemisphere'])
        else:  # else encode the labels
            # encode
            le.fit(DF_DATASET['hemisphere'])
            DF_DATASET['hemisphere'] = le.transform(DF_DATASET['hemisphere'])
    return DF_DATASET

def h_cluster_data(DF_DATASET, n_clusters=None):
    # encode the labels
    DATASET = encode_labels(DF_DATASET, True)
    # Scale the Data
    scaled_data = scale_data(DF_DATASET)
    # instantiate the Clustering Model
    model = AgglomerativeClustering(distance_threshold=0, n_clusters = n_clusters)
    # Fit and transform the data
    clusters = model.fit_predict(scaled_data)
    DF_DATASET['cluster'] = clusters
    DF_DATASET = encode_labels(DF_DATASET, False)
    return DF_DATASET

def k_cluster_data(DF_DATASET, n_clusters=None):
    # encode the labels
    DATASET = encode_labels(DF_DATASET, True)
    # Scale the Data
    scaled_data = scale_data(DF_DATASET)
    # instantiate the Clustering Model
    model = KMeans(n_clusters=n_clusters)
    # fit and store the predictions
    clusters = model.fit_predict(scaled_data)
    # add a new column with the cluster values
    DF_DATASET['cluster'] = clusters
    # un encode the labels on the Dataframe
    DF_DATASET = encode_labels(DF_DATASET, False)
    return DF_DATASET, model

def k_cluster_data2(DF_DATASET, n_clusters=None):
    # lets prep the list
    year_list_n = {}
    year_list_s = {}
    # this puts each year into its own list in the year_list dict
    for year in range(1979, 2018):
        item = DF_DATASET[DF_DATASET['year'] == year]
        # This adds a dict inside of the year val
        year_list_n[year] = {}
        year_list_s[year] = {}
        # then adds the values to the dict
        # counts up to 366(for leap years...)
        for dayofyear in range(1, 367):
            n_it = item[item['dayofyear'] == dayofyear]
            # empty vals for use in try statementlater
            s_it_n = None
            n_it_n = None
            try:
                # this will throw an exception if val is None
                n_it_n = n_it['n_extent'].values.item()
            except:
                # so we want to store it as None anyways
                n_it_n = None
            try:
                s_it_n = n_it['s_extent'].values.item()
            except:
                s_it_n = None
            year_list_n[year][dayofyear] = n_it_n
            year_list_s[year][dayofyear] = s_it_n
    # create dataframe objs from dicts
    df_yls = DataFrame(data=year_list_s)
    df_yln = DataFrame(data=year_list_n)
    # transpose vals
    df_yls = df_yls.transpose()
    df_yln = df_yln.transpose()
    # reset the indexes
    df_yln.reset_index(inplace=True)
    df_yls.reset_index(inplace=True)
    # create column name list for new dataframe
    column_names = ['year']
    for i in range(1, 367):
        column_names.append(i)
    # set column names on dataframes
    df_yln.columns = column_names
    df_yls.columns = column_names
    # Now impute the columns
    # This will fill in any NaN vals with the mean value from the column
    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    # fills the missing values
    df_yln = imputer.fit_transform(df_yln)
    df_yls = imputer.fit_transform(df_yls)
    # recreate the dataframes
    df_yln = DataFrame(df_yln)
    df_yls = DataFrame(df_yls)
    # reset the column names
    df_yln.columns = column_names
    df_yls.columns = column_names
    # Now for the Kmeans for the north...
    kmean_n = KMeans(n_clusters=n_clusters)
    kmean_n.fit(df_yln)
    clusters = kmean_n.predict(df_yln)
    df_yln['clusters'] = clusters
    # ...and for the south
    kmean_s = KMeans(n_clusters=n_clusters)
    kmean_s.fit(df_yls)
    clusters = kmean_s.predict(df_yls)
    df_yls['clusters'] = clusters
    # return both data frames with clusters as well as kmeans
    return df_yln, df_yls ,kmean_n, kmean_s

# DATA_PREP #
#############
def prep_data(source):

    # PREPARE DATASET
    df = pd.read_csv(source)
    print('CSV Read')
    # rename columns
    df.columns = ['year', 'month','day', 'extent','missing','source_data','hemisphere']
    # remove unnecessary columns, i.e. source data
    df = df.drop(columns=['source_data'])
    # split the data frames
    df_n = df[df['hemisphere'] =='north'] # all north extents
    df_s = df[df['hemisphere'] =='south'] # all south extents
    # merge the dataframes together
    df_n.columns = ['year','month','day','n_extent','n_missing', 'df_n']
    df_s.columns = ['year','month','day','s_extent','s_missing', 'df_s']
    DF_DATASET = df_n.merge(df_s)
    DF_DATASET = DF_DATASET.drop(columns=['df_n', 'df_s'])
    # Now add a column with the date string
    DF_DATASET['date'] = df[['month','day','year']].astype(str).agg('-'.join, axis=1)
    df['date'] = df[['month','day','year']].astype(str).agg('-'.join, axis=1)
    # create dayof the year column

    DF_DATASET['dayofyear'] = hlp.day_of_year_getter(DF_DATASET['date'])
    return DF_DATASET, df

def get_avgs(df):
    # group rows by year and month and average the corresponding values
    avg_df = df.groupby(['year','month','hemisphere']).agg([np.average])
    # set index
    avg_df.index = avg_df.index.set_names(['year', 'month', 'hemisphere'])
    # reset index to fill in vals
    avg_df.reset_index(inplace=True)
    # rename cols
    avg_df.columns = ['year', 'month', 'hemisphere', 'dayavg', 'extentavg', 'missingavg']
    # remove nulls if any
    avg_df.dropna()
    return avg_df

def prep_bydoy(DF_DATASET):
    # lets prep the list
    year_list = {}
    year_list_s = {}
    # this puts each year into its own list in the year_list dict
    for year in range(1979, 2015, 5):
        item = DF_DATASET[DF_DATASET['year'] == year]
        year_list[year] = {}
        year_list_s[year] = {}
        for dayofyear in range(1, 366):
            n_it = item[item['dayofyear'] == dayofyear]

            s_it_n = None
            n_it_n = None
            try:
                n_it_n = n_it['n_extent'].values.item()
            except:
                n_it_n = None
            try:
                s_it_n = n_it['s_extent'].values.item()
            except:
                s_it_n = None
            year_list[year][dayofyear] = n_it_n
            year_list_s[year][dayofyear] = s_it_n

    # output is 1979, 1980, etc
    df_yls = pd.DataFrame(data=year_list_s).transpose()
    df_yln = pd.DataFrame(data=year_list).transpose()


    return df_yls, df_yln

# DATA_VIS #
############

def plotly_scatter_plot(_month, _day, _year, DF_DATASET, N_L_REGR, S_L_REGR):
    day_of_year = hlp.single_day_oy_getter(_year,_month,_day)
    # get days to plot
    add_subtract_list = hlp.get_add_subtract_days(_month, _day, _year, 15)
    # get days from dataset for plot
    df_data = DF_DATASET.loc[(DF_DATASET['dayofyear'].isin(add_subtract_list))]
    X = df_data[['dayofyear']]
    y_n = df_data[['n_extent']]
    s_n = df_data[['s_extent']]
    # get predictions
    n_prediction = N_L_REGR.predict([[_year, _month, _day ,day_of_year]])
    s_prediction = S_L_REGR.predict([[_year, _month, _day ,day_of_year]])
    # for hover templates
    template_str = '<b>Extent</b>: %{y:.2f}' + '<br><b>Day of Year</b>: %{x}<br>' + '<b>Date</b>: %{text}'
    # create scatter plot for northern historical data
    trace = go.Scatter(
        x=df_data['dayofyear'],
        y=df_data['n_extent'],
        opacity=0.6,
        hovertemplate =template_str,
        text=df_data['date'],
        name="<i>Northern Historical</i>",
        mode='markers')
    # create scatter plot for southern historical data
    trace2 = go.Scatter(
        x=df_data['dayofyear'],
        y=df_data['s_extent'],
        opacity=0.6,
        hovertemplate=template_str,
        text=df_data['date'],
        name="<i>Southern Historical</i>",
        mode='markers')
    # create scatter plots for predictions
    import numpy as np
    n_prediction = go.Scatter(
        x=np.array([day_of_year]),
        y=n_prediction[0],
        mode='markers',
        marker = dict(size=[20],color='#3136cc'),
        visible=True,
        name="<b>Northern Prediction</b>")
    s_prediction = go.Scatter(
        x=np.array([day_of_year]),
        y=s_prediction[0],
        mode='markers',
        marker = dict(size=[20], color='#db432c'),
        visible=True,
        name="<b>Southern Prediction</b>")
    # create list of plots
    data = [trace, trace2, n_prediction, s_prediction]
    # create json data for passing to html output
    graph_json = json.dumps(data, cls=plotly.utils.PlotlyJSONEncoder)
    return graph_json


# creates a violin plot to be used for the data visualizations
def plotly_violin_plot(start, end, rate, df):
    yr_list = []
    # generate a list with the start and end dates to be plotted.
    for i in range(start, end, rate):
        yr_list.append(i)

    data_fr = df[df['year'].isin(yr_list)]

    left = go.Violin(x=data_fr['year'][ data_fr['hemisphere'] == 'north' ],
                        y=df['extent'][ df['hemisphere'] == 'north' ],
                        legendgroup='north', scalegroup='north', name='north',
                        side='negative',
                        line_color='blue')
    right = go.Violin(x=data_fr['year'][ data_fr['hemisphere'] == 'south' ],
                        y=df['extent'][ df['hemisphere'] == 'south' ],
                        legendgroup='south', scalegroup='south', name='south',
                        side='positive',
                        line_color='orange')

    data = [left, right]
    graph_json = json.dumps(data, cls=plotly.utils.PlotlyJSONEncoder)
    return graph_json


def plotly_heatmap(avg_df, hemisphere):
    template_str = '<b>Extent</b>: %{z:.2f}' + '<br><b>Year</b>: %{x}<br>' + '<b>Month</b>: %{y}'
    # takes a string as hemisphere and returns a heatmap JSON
    avgs = avg_df[avg_df['hemisphere'] == hemisphere]
    heat = go.Heatmap(x=avgs['year'],
                      y=avgs['month'],
                      z=avgs['extentavg'],
                      hovertemplate=template_str,
                      colorscale='Viridis',
                      name='<b>'+hemisphere.capitalize()+'</b>')
    data = [heat]
    graph_json = json.dumps(data, cls=plotly.utils.PlotlyJSONEncoder)
    return graph_json

def plotly_kmeans_scatter(DF_DATASET):
    # this will plot the Dataset with the kmeans cluster from data that was
    # transformed so that each year was on an evenly matched 366 day year of values
    tr_df = DF_DATASET.transpose()
    tr_df.columns = tr_df.iloc[0]
    tr_df.drop(['year'], inplace=True)
    tr_df.reset_index(inplace=True)
    tr_df.drop(columns=['index'], inplace=True)

    charts = []
    for i in range(1979, 2018):
        X = tr_df[i].index[:-1]
        Y = tr_df[i][:-1]
        cluster = int(tr_df[i][366].item())
        colrs = ['#0d0887', '#46039f', '#7201a8', '#9c179e']
        charts.append(
            go.Scatter(x=X,
                       y=Y,
                       marker=dict(
                           color=colrs[cluster],
                           # colorscale='Viridis'
                       ),
                       legendgroup=cluster,
                       opacity=0.6,
                       mode='lines',
                       text=i,
                       name=i

                       )
        )
    data = charts
    graph_json = json.dumps(data, cls=plotly.utils.PlotlyJSONEncoder)
    return graph_json

# Create a dataframe with the DF_DATASET dataframe and averages made up from that
def plotly_bar_plots(DF_DATASET, s):
    df_year = DF_DATASET
    df_year_agg = df_year.groupby(['year','month']).agg([np.average])
    df_year_agg.index = df_year_agg.index.set_names(['year', 'month'])
    # reset index to fill in vals
    df_year_agg.reset_index(inplace=True)
    # rename cols
    df_year_agg.columns = ['year','month','dayavg','n_ext','n_miss','s_ext','s_miss','dayoyavg','cls']
    df_year_agg.head()

    years = [1980,1985,1990, 1995, 2000, 2005, 2010, 2015, 2018]
    data = []
    data2 = []
    # bar plot of extent values for each month of each year
    for year in years:
        df_year = df_year_agg[df_year_agg['year']==year]
        north = str(year) + ' North'
        data.append(go.Bar(
            y=df_year['n_ext'],
            x=df_year['month'],
            name=north
        ))
        south = str(year) + ' South'
        data2.append(go.Bar(
            y=df_year['s_ext'],
            x=df_year['month'],
            name=south
        ))
    data = go.Figure(data=data)
    data2 = go.Figure(data=data2)
    n_graph_json = json.dumps(data, cls=plotly.utils.PlotlyJSONEncoder)
    s_graph_json = json.dumps(data2, cls=plotly.utils.PlotlyJSONEncoder)
    return n_graph_json, s_graph_json

# HELPERS #
###########

def day_of_year_getter(date_vals):
    # takes in an object with a list stored in the values attribute
    str_list = date_vals.values
    rt_list = []
    for date_str in str_list:
        date_format = '%m-%d-%Y'
        current_date = datetime.strptime(date_str, date_format)
        # day_delta = current_date - START_DATE
        day_delta = current_date.timetuple().tm_yday
        rt_list.append(day_delta)
    return rt_list

def date_transformer(date_str):
    # this returns a date time obj
    new_date
    try:
        new_date = parse(date_str)
    except:
        format = '%Y-%m-%d'
        new_date = datetime.strptime(date_str, format)
    return new_date

def single_day_oy_getter(y, m, d):
    # takes in month day and year vals
    date_ = datetime(y, m, d)
    day_delta = date_.timetuple().tm_yday
    return day_delta

def day_of_year_add_subtract(date_str, _days):
    # takes in date string
    date_format = '%m-%d-%Y'
    current_date = datetime.strptime(date_str, date_format)
    rt_list = []
    plus_day = current_date + timedelta(days=_days)
    rt_list.append(plus_day.timetuple().tm_yday)
    minus_day = current_date - timedelta(days=_days)
    rt_list.append(minus_day.timetuple().tm_yday)
    # returns a list with 0 = plus_day and 1 = minus_day
    return rt_list

def get_add_subtract_days(month_, day_, year_, _days):
    # this will return a list of daysofyear for plotting and linear regression
    date_format = '%m-%d-%Y'
    current_date = datetime(year_,month_, day_)
    rt_list = []
    # adds current_date
    rt_list.append(current_date.timetuple().tm_yday)
    for d in range(1, _days+1):
        rt_list.append((current_date + timedelta(days=d)).timetuple().tm_yday)
        rt_list.append((current_date - timedelta(days=d)).timetuple().tm_yday)
    return rt_list

class Dummy:
    def __init__(self, _val):
        self.values = []
        self.values.append(_val)


In [ ]:
# Now lets get the datasets built
DF_DATASET, df = prep_data('app/dataset/seaice.csv')
n_clusters = 4
DF_DATASET, kmeans_model = k_cluster_data(DF_DATASET, n_clusters)
year_col_df_north, year_col_df_south, kmeans_n, kmeans_s = k_cluster_data2(DF_DATASET, n_clusters)
N_L_REGR, S_L_REGR = get_linear_pred(DF_DATASET)
# get Avgs
avg_df = get_avgs(df)
avg_df, avg_kmeans = k_cluster_data(avg_df, n_clusters)

print('#######' * 3, 'DF_DATASET', '#######' * 3)
print(DF_DATASET.head())
print('#######' * 3, 'Averages By Month', '#######' * 3)
print(avg_df.head())


In [ ]:
# Now for the Data Visualizations

